<a href="https://colab.research.google.com/github/unkeyol-pbl/PBL/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
"""mediapipe 설치"""

# MediaPipe와 호환되는 numpy 버전을 강제 재설치
!pip install numpy==1.26.4 mediapipe==0.10.21 --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.2/113.2 kB 3.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 85.8 MB/s

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 55.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.21 requires numpy<2, but you have numpy 2.2.6 which is incompatible.
gradio 5.50.0 requires pillow<12.0,>=8.0, but you have pillow 12.0.0 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 whi

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import mediapipe as mp
from ultralytics import YOLO
from tensorflow.keras.models import load_model
from google.colab import drive
import os

# 1. 구글 드라이브 마운트
drive.mount('/content/drive')

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# =========================================================
# [필수] Stage 1 모델 아키텍처 정의 (PyTorch 모델 로드용)
# =========================================================

JOINT_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 7), (0, 4), (4, 5), (5, 6), (6, 8),
    (9, 10), (11, 12), (11, 13), (13, 15), (12, 14), (14, 16)
]

def get_adjacency_matrix(num_joints=33):
    A = torch.eye(num_joints)
    for i, j in JOINT_CONNECTIONS:
        A[i, j] = 1
        A[j, i] = 1
    D_inv_sqrt = torch.diag(torch.pow(A.sum(1), -0.5))
    return D_inv_sqrt @ A @ D_inv_sqrt

class GraphConv(nn.Module):
    def __init__(self, in_features, out_features, A):
        super().__init__()
        self.A = A
        self.fc = nn.Linear(in_features, out_features)

    def forward(self, x):
        Ax = torch.einsum('ij,bjc->bic', self.A, x)
        return F.relu(self.fc(Ax))

class GCN_LSTM(nn.Module):
    def __init__(self, num_joints=33, in_features=4, gcn_hidden=64, lstm_hidden=128):
        super().__init__()
        self.A = get_adjacency_matrix(num_joints).to(torch.float32).to(device)
        self.gcn1 = GraphConv(in_features, gcn_hidden, self.A)
        self.gcn2 = GraphConv(gcn_hidden, gcn_hidden, self.A)
        self.lstm = nn.LSTM(gcn_hidden * num_joints, lstm_hidden, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(lstm_hidden * 2, 1)

    def forward(self, x):
        B, T, J, C = x.shape
        feats = []
        for t in range(T):
            xt = x[:, t, :, :]
            h = self.gcn1(xt)
            h = self.gcn2(h)
            feats.append(h.view(B, -1))
        x_seq = torch.stack(feats, dim=1)
        _, (hn, _) = self.lstm(x_seq)
        hn_cat = torch.cat((hn[0], hn[1]), dim=1)
        return self.fc(hn_cat).squeeze(1)

# =========================================================
# [실행] 모델 불러오기
# =========================================================

STAGE1_MODEL_PATH = "/content/drive/MyDrive/elderlycare_violence/model/best_model_GCN-BILSTM.pth"
STAGE2_MODEL_PATH = "/content/drive/MyDrive/elderlycare_violence/model/skeleton_pinch_v3.keras"

print("\n--- 모델 로드 시작 ---")

# 1. Stage 1 (전신 폭행) 모델 로드
try:
    print("Loading Stage 1 (GCN-BiLSTM + YOLO + Pose)...")

    # YOLO & MediaPipe 초기화
    yolo_model = YOLO("yolov8n.pt")
    pose_detector = mp.solutions.pose.Pose(
        static_image_mode=False, model_complexity=2, min_detection_confidence=0.2
    )

    # GCN 모델 초기화 및 가중치 로드
    model_stage1 = GCN_LSTM().to(device)
    if os.path.exists(STAGE1_MODEL_PATH):
        model_stage1.load_state_dict(torch.load(STAGE1_MODEL_PATH, map_location=device))
        model_stage1.eval() # 평가 모드로 전환 (Dropout 등 비활성화)
        print("Stage 1 모델 가중치 로드 성공!")
    else:
        print(f"오류: Stage 1 파일을 찾을 수 없습니다. 경로: {STAGE1_MODEL_PATH}")

except Exception as e:
    print(f" Stage 1 로드 중 에러 발생: {e}")

# 2. Stage 2 (손 꼬집기) 모델 로드
try:
    print("Loading Stage 2 (Hand Pinch DNN)...")

    # MediaPipe Hands 초기화
    hands_detector = mp.solutions.hands.Hands(
        static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5
    )

    # Keras 모델 로드
    if os.path.exists(STAGE2_MODEL_PATH):
        model_stage2 = load_model(STAGE2_MODEL_PATH)
        print(" Stage 2 모델 로드 성공!")
    else:
        print(f"오류: Stage 2 파일을 찾을 수 없습니다. 경로: {STAGE2_MODEL_PATH}")

except Exception as e:
    print(f"Stage 2 로드 중 에러 발생: {e}")

print("\n------------------------------------------------")
if 'model_stage1' in locals() and 'model_stage2' in locals():
    print("모든 모델이 정상적으로 메모리에 로드되었습니다.")
else:
    print("일부 모델 로드에 실패했습니다. 경로를 다시 확인해주세요.")

In [ ]:
from tqdm.notebook import tqdm

# =========================================================
# 테스트할 영상 경로를 입력
# =========================================================
INPUT_VIDEO_PATH = '/content/drive/MyDrive/hand_project/enhanced_video/MOVI0004_enhanced.mp4'

# 결과 저장 설정
original_filename = os.path.basename(INPUT_VIDEO_PATH).split('.')[0]
OUTPUT_VIDEO_PATH = os.path.join(BASE_DIR, f"{original_filename}_TEST_FINAL.mp4")
PREDICTIONS_SAVE_PATH = os.path.join(BASE_DIR, f"{original_filename}_predictions_final.npy")
CAPTURED_FRAMES_DIR = '/content/drive/MyDrive/hand_project/Captured_Frames_Final'
os.makedirs(CAPTURED_FRAMES_DIR, exist_ok=True)

# ---------------------------------------------------------
# 파라미터 튜닝
# ---------------------------------------------------------
WEIGHT_DNN = 0.5
WEIGHT_STGCN = 0.5
SEQ_LEN = 30
FRAME_SKIP_INTERVAL = 3
PINCH_DISPLAY_DURATION = 15

DISTANCE_THRESHOLD_GRAB = 0.15  #  0.15 (팔 두께 허용)
MIN_CLOSING_RATE = 0.003        # 조금만 닫혀도 반응하도록 민감도 조정
CLS_ID_FOR_HAND = 0             # 사람(Person) 클래스

# ---------------------------------------------------------
# 영상 분석
# ---------------------------------------------------------
cap = cv2.VideoCapture(INPUT_VIDEO_PATH)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (width, height))

print(f"\n🎬 분석 시작: {total_frames} 프레임 (Grab 감지 강화 모드)")
pbar = tqdm(total=total_frames // FRAME_SKIP_INTERVAL)

stgcn_sequence = []
all_predictions = []
frame_counter = 0
prev_dist = None

# 결과 유지용
pinch_display_counter = 0
last_label_text = ""
last_color = (255, 0, 0)
last_box = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    frame_counter += 1
    if frame_counter % FRAME_SKIP_INTERVAL != 0:
        out.write(frame)
        all_predictions.append(0)
        continue

    # [1] YOLO 탐지
    yolo_results = yolo_model(frame, verbose=False, device='cpu')
    hand_boxes = []
    is_object_present = False # 내 손 외에 다른 사람이나 물체가 있는지 확인

    # YOLO에서 감지된 사람(Person)의 수 확인
    person_count = 0
    for r in yolo_results:
        for i, cls_id in enumerate(r.boxes.cls.cpu().numpy().astype(int)):
            if cls_id == CLS_ID_FOR_HAND:
                hand_boxes.append(r.boxes.xyxy.cpu().numpy().astype(int)[i])
                person_count += 1

    # 사람이 2명 이상이면 Logic 활성화
    if person_count >= 2:
        is_object_present = True

    # [2] Hand Process
    current_pred = 0
    is_pinch_detected_now = False

    if hand_boxes:
        for box in hand_boxes:
            x1, y1, x2, y2 = box
            cropped = frame[y1:y2, x1:x2]
            if cropped.size == 0: continue

            mp_res = hands.process(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))

            if mp_res.multi_hand_landmarks:
                for hand_idx, lm_obj in enumerate(mp_res.multi_hand_landmarks):
                    if hand_idx >= 2: break

                    kp_list = []
                    lm_xyz = []
                    for lm in lm_obj.landmark:
                        kp_list.extend([lm.x, lm.y, lm.z])
                        lm_xyz.append([lm.x, lm.y, lm.z])
                    lm_xyz = np.array(lm_xyz)

                    # DNN 입력
                    final_kp_dnn = np.zeros(126)
                    if hand_idx == 0: final_kp_dnn[:len(kp_list)] = kp_list[:126]

                    # 거리 계산 (엄지-검지)
                    thumb = lm_xyz[4]
                    index = lm_xyz[8]
                    dist = np.linalg.norm(thumb - index)

                    # ST-GCN 입력
                    dist_feat = np.full((NUM_JOINTS, 1), dist if dist >= 0.25 else dist * 0.2)
                    current_keypoints_frame = np.zeros((2, NUM_JOINTS, 4), dtype=np.float32)
                    current_keypoints_frame[hand_idx] = np.concatenate((lm_xyz, dist_feat), axis=1)

                    # 시각화
                    mp_drawing.draw_landmarks(cropped, lm_obj, mp_hands.HAND_CONNECTIONS)

                    # 예측
                    prob_dnn = dnn_model.predict(final_kp_dnn.reshape(1, -1), verbose=0)[0][0]

                    prob_stgcn = 0.0
                    stgcn_sequence.append(current_keypoints_frame)
                    if len(stgcn_sequence) > SEQ_LEN: stgcn_sequence.pop(0)

                    if stgcn_model and len(stgcn_sequence) == SEQ_LEN:
                        # ST-GCN 추론
                        seq_arr = np.array(stgcn_sequence, dtype=np.float32).reshape(SEQ_LEN, 2, NUM_JOINTS, 4)
                        input_data = np.transpose(seq_arr, (3, 0, 2, 1))
                        input_data = np.expand_dims(input_data, axis=0)
                        wrist_id = 0
                        for m in range(2):
                            input_data[0, :3, :, :, m] -= input_data[0, :3, :, wrist_id, m][:, :, None]
                        velocity = np.zeros((1, 3, SEQ_LEN, NUM_JOINTS, 2), dtype=np.float32)
                        velocity[:, :, 1:, :, :] = input_data[:, :3, 1:, :, :] - input_data[:, :3, :-1, :, :]
                        final_input = np.concatenate((input_data, velocity), axis=1)
                        input_tensor = torch.tensor(final_input, dtype=torch.float32).to(device)
                        with torch.no_grad():
                            out_stgcn = stgcn_model(input_tensor)
                            prob_stgcn = F.softmax(out_stgcn, dim=1)[0, 1].item()
                    else:
                        prob_stgcn = prob_dnn

                    final_score = (prob_dnn * WEIGHT_DNN) + (prob_stgcn * WEIGHT_STGCN)

                    # Closing Logic
                    is_closing = False
                    if prev_dist is not None and (prev_dist - dist) > MIN_CLOSING_RATE:
                        is_closing = True
                    prev_dist = dist

                    # ---  판정 로직 수정 ---
                    # 1. AI가 확신할 때
                    if final_score > 0.6:
                        is_pinch_detected_now = True
                        label_text = f"PINCH V4 ({final_score*100:.0f}%)"
                        color = (0, 255, 0)

                    # 2. Logic (움켜잡기): 물체(사람)가 있고 + 손이 닫히며 + 거리가 적당히 가까울 때(0.15)
                    elif dist < DISTANCE_THRESHOLD_GRAB and is_closing and is_object_present:
                        is_pinch_detected_now = True
                        label_text = "GRAB (Arm)" # 명칭 변경
                        color = (0, 255, 255) # 노란색

            # 시각화된 손 다시 붙여넣기
            frame[y1:y2, x1:x2] = cropped
            last_box = box

    # --- 결과 표시 ---
    if is_pinch_detected_now:
        pinch_display_counter = PINCH_DISPLAY_DURATION
        last_label_text = label_text
        last_color = color
        current_pred = 1
        # 중요 장면 캡처
        cv2.imwrite(os.path.join(CAPTURED_FRAMES_DIR, f"Grab_Action_{frame_counter}.jpg"), frame)

    if pinch_display_counter > 0:
        if last_box is not None:
            x1, y1, x2, y2 = last_box
            # 크고 굵게 표시
            cv2.rectangle(frame, (x1, y2-80), (x1+350, y2), last_color, -1)
            cv2.putText(frame, last_label_text, (x1+10, y2-20), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0,0,0), 4)
        pinch_display_counter -= 1

    out.write(frame)
    all_predictions.append(current_pred)
    pbar.update(1)

cap.release()
out.release()
pbar.close()
np.save(PREDICTIONS_SAVE_PATH, np.array(all_predictions))

print(f"\n✨ 테스트 완료!")
print(f"📄 영상: {OUTPUT_VIDEO_PATH}")
print(f"📊 결과: {PREDICTIONS_SAVE_PATH}")